In [51]:
import h5py
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Locally get vars to accommodate different environments
try:
    from env_vars import *
except:
    print(
        'No env_vars file found. Using default values. Make a "env_vars.py" file to change them.'
    )
    intra_train_path = "./data/Cross/train/"
    intra_test_path = "./data/Cross/test1/"

No env_vars file found. Using default values. Make a "env_vars.py" file to change them.


In [52]:
# from google.colab import drive

# drive.mount("/content/gdrive")

In [53]:
# Used to get the name of the dataset, and by extension, the label of the action being performed
def getdatasetname(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split("/")[-1]

    temp = filename_without_dir.split("_")[:-1]

    datasetname = "_".join(temp)

    return datasetname

In [54]:
# Used to get the labels of data using filenames
def get_label(filename):
    if "rest" in filename:
        label = 0

    elif "math" in filename:
        label = 1

    elif "memory" in filename:
        label = 2
    elif "motor" in filename:
        label = 3

    return label

In [55]:
def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)

    return dataset, labels

MinMax Scaling for sensor data


In [56]:
def scale(matrix):
    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(matrix)

    scaled_data = scaler.transform(matrix)

    return scaled_data

Downsampling of data, refer to file called "Notebook.ipynb" to see how it works


In [57]:
def downsample(dataset, frequency):
    downsampled_dataset = []

    for i in range(0, dataset.shape[1], 2034):
        second = dataset[:, i : i + 2034]
        subsample = []

        for j in range(0, 2034, int(2034 / frequency)):
            if j < second.shape[1]:
                measurement = second[:, j]
                subsample.append(measurement)

        downsampled_dataset.extend(subsample)

    return np.array(downsampled_dataset).T

Model setup


In [58]:
# TensorFlow and tf.keras
import tensorflow as tf

print(tf.__version__)

2.15.0


In [59]:
def load_data(dir_path):
    data = []
    labels = []

    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = os.path.join(dir_path, filename)
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]

                train_meg = downsample(matrix, 113)
                train_meg = scale(train_meg)
                # flattened_meg = np.array(
                #     train_meg.flatten()
                # )  # The data is flattened to change the meg data from shape 248 x frequency to a 1D array (result: a lot of inupt params)

                data.append(train_meg)
                labels.append(label)

    return np.array(data), np.array(labels)

All training data is loaded into memory and fitted to the model along with the training lables


In [60]:
X_train, y_train = load_data(intra_train_path)

In [61]:
print(X_train.shape)
print(y_train.shape)

(64, 248, 17812)
(64,)


In [62]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv1D(
            32,
            kernel_size=3,
            activation="relu",
            input_shape=(X_train.shape[1], X_train.shape[2]),
        ),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        # tf.keras.layers.Dense(128),
        tf.keras.layers.LSTM(
            248, return_sequences=True
        ),  # Adjust the number of LSTM units
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(4),
    ]
)

In [63]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [64]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
2/2 [==============================] - 22s 2s/step - loss: 1.6112 - accuracy: 0.2969
Epoch 2/10
2/2 [==============================] - 2s 1s/step - loss: 1.4925 - accuracy: 0.2031
Epoch 3/10
2/2 [==============================] - 3s 2s/step - loss: 1.3657 - accuracy: 0.2500
Epoch 4/10
2/2 [==============================] - 3s 1s/step - loss: 1.3473 - accuracy: 0.3906
Epoch 5/10
2/2 [==============================] - 2s 1s/step - loss: 1.3393 - accuracy: 0.2969
Epoch 6/10
2/2 [==============================] - 2s 1s/step - loss: 1.2844 - accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 3s 1s/step - loss: 1.1955 - accuracy: 0.5000
Epoch 8/10
2/2 [==============================] - 2s 1s/step - loss: 1.2950 - accuracy: 0.3906
Epoch 9/10
2/2 [==============================] - 2s 1s/step - loss: 1.1557 - accuracy: 0.5000
Epoch 10/10
2/2 [==============================] - 2s 1s/step - loss: 1.2687 - accuracy: 0.3750


In [65]:
X_test, y_test = load_data(intra_test_path)

In [66]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print("\nTest accuracy:", test_acc)

1/1 - 7s - loss: 1.1201 - accuracy: 0.5000 - 7s/epoch - 7s/step

Test accuracy: 0.5
